In [17]:
import pandas as pd

moves = pd.read_csv("data/moves.csv")
text = moves[["Name", "Effect"]]
combined_move = text.apply(lambda x: x["Name"] + " " + x["Effect"], axis = 1).tolist()
text = ''.join(str(elem) for elem in combined_move)

In [29]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import io



#path = keras.utils.get_file(
#    "nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt"
#)
#with io.open(path, encoding="utf-8") as f:
#    text = f.read().lower()



#text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
    
    
    
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(64),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)





def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


epochs = 40
batch_size = 128



Corpus length: 82111
Total chars: 70
Number of sequences: 27357


In [ ]:
for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        #sentence = "Surging Strikes"
        print('...Generating with seed: "' + sentence + '"')

        for i in range(100):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()

214/214 [==============================] - 5s 19ms/step - loss: 2.7839

Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: "c Laser The user shoots powerful lasers "


Exception ignored in: <function ScopedTFGraph.__del__ at 0x7f8cc8171af0>
Traceback (most recent call last):
  File "/Users/ArjunPatel/opt/anaconda3/envs/Speeko_Testing/lib/python3.8/site-packages/tensorflow/python/framework/c_api_util.py", line 58, in __del__
    self.deleter(self.graph)
AttributeError: deleter


...Generated:  and the target the target the target target target target als als ats als ans the target the user al

...Diversity: 0.5
...Generating with seed: "c Laser The user shoots powerful lasers "
...Generated:  als ald ats the user s bery the target ats atd thit as atatstamas the user s whit its the target the

214/214 [==============================] - 4s 19ms/step - loss: 1.7724

Generating text after epoch: 1
...Diversity: 0.2
...Generating with seed: "he user moves after the target, this att"
...Generated:  acks the target its attacks the target its attacks the target its the target with stattack the targe

...Diversity: 0.5
...Generating with seed: "he user moves after the target, this att"
...Generated:  acked. This move the target ind chaske twattat. This may also damang the target with sping. This may

214/214 [==============================] - 4s 19ms/step - loss: 1.5458

Generating text after epoch: 2
...Diversity: 0.2
...Generating with seed: ". If this move is used e

In [15]:
path = keras.utils.get_file(
    "nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt"
)
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()



text = text.replace("\n", " ")  # We remove newlines chars for nicer display